Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

1行に1記事の情報がJSON形式で格納される
各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
ファイル全体はgzipで圧縮される
以下の処理を行うプログラムを作成せよ．

### 20. JSONデータの読み込みPermalink
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [ ]:
import pandas as pd
df = pd.read_json('../input/jawiki-country.json', lines=True)
uk = df.text[df.title=='イギリス'].values
print(uk)

### 21. カテゴリ名を含む行を抽出Permalink
記事中でカテゴリ名を宣言している行を抽出せよ．

In [ ]:
import pandas as pd
import re
df = pd.read_json('../input/jawiki-country.json', lines=True)
uk = df.text[df.title=='イギリス'].values
for line in uk[0].splitlines():
    if re.search('Category', line):
        print(line)

### 22. カテゴリ名の抽出Permalink
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [ ]:
import pandas as pd
import re
df = pd.read_json('../input/jawiki-country.json', lines=True)
uk = df.text[df.title=='イギリス'].values

for line in uk[0].splitlines():
    if re.search('Category', line):
        line = line.replace('[', '').replace(']', '').replace('Category:', '').replace('|元', '').replace('|*', '')
        print(line)

### 23. セクション構造Permalink
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [ ]:
import pandas as pd
import re
df = pd.read_json('../input/jawiki-country.json', lines=True)
uk = df.text[df.title=='イギリス'].values

for line in uk[0].splitlines():
    if re.search("==+", line):
        section_level = int(line.count('=') / 2 -1)
        print(line.replace('=', ''), section_level)

#a = 'abcdfgavdfkhfdaoiejrdlsa'
#a.count('a')
#[OUT] 4

### 24. ファイル参照の抽出Permalink
記事から参照されているメディアファイルをすべて抜き出せ．

In [ ]:
import pandas as pd
import re
df = pd.read_json('../input/jawiki-country.json', lines=True)
uk = df.text[df.title=='イギリス'].values
for line in uk[0].splitlines():
    file = re.findall('File|ファイル:(.+?)\|', line)
    if file:
        print(file[0])
        
#(.+?)  ()で文字を一つのグループにまとめる

### 25. テンプレートの抽出Permalink
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [ ]:
import pandas as pd
import re
df = pd.read_json('../input/jawiki-country.json', lines=True)
uk = df.text[df.title=='イギリス'].values

basic_info = {}
for line in uk[0].splitlines():
    file = re.search('\|(.+?)\s=\s*(.+)', line)
    if file:
        basic_info[file[1]] = file[2]
basic_info
#\s 空白表現(\S 全ての非空白表現)

### 26. 強調マークアップの除去Permalink
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
import pandas as pd
import re
df = pd.read_json('../input/jawiki-country.json', lines=True)
uk = df.text[df.title=='イギリス'].values

basic_info = {}
for line in uk[0].splitlines():
    r = re.search('\|(.+?)\s=\s*(.+)', line)
    if r:
        basic_info[r[1]] = r[2]
    print(re.sub('\'{2, }(.+?)\'{2, }', '\\1', line))
print(basic_info)

#{n, } 最小桁数の指定 ex. a{3, } -> aaa, aaaa, aaaaa, ....  

### 27. 内部リンクの除去Permalink
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
import pandas as pd
import re
pattern = re.compile('\|(.+?)\s=\s*(.+)')
pattern1 = re.compile('\'{2,}(.+?)\'{2,}')
pattern2 = re.compile('\[\[(.+?)\]\]')
df = pd.read_json('../input/jawiki-country.json', lines = True)
uk = df[df['title']=='イギリス'].text.values
lines = uk[0]
lines = re.sub(pattern1,'\\1', lines)
lines = re.sub(pattern2,'\\1', lines)
ls = lines.split('\n')
d = {}
for line in ls:
    r = re.search(pattern, line)
    if r:
        d[r[1]]=r[2]
print (d)

### 28. MediaWikiマークアップの除去Permalink
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [ ]:
import pandas as pd
import re
pattern = re.compile('\|(.+?)\s=\s*(.+)')
pattern1 = re.compile('\'{2,}(.+?)\'{2,}')
pattern2 = re.compile('\[\[(.+?)\]\]')
pattern3 = re.compile('<[br|ref][^>]*?>.+?<\/[br|ref][^>]*?>')
df = pd.read_json('../input/jawiki-country.json', lines = True)
uk = df[df['title']=='イギリス'].text.values
lines = uk[0]
lines = re.sub(pattern1,'\\1', lines)
lines = re.sub(pattern2,'\\1', lines)
lines = re.sub(pattern3,'', lines)
ls = lines.split('\n')
d = {}
for line in ls:
    r = re.search(pattern, line)
    if r:
        d[r[1]]=r[2]
print (d)

### 29. 国旗画像のURLを取得するPermalink
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [2]:
import pandas as pd
import re
import requests
pattern = re.compile('\|(.+?)\s=\s*(.+)')
df = pd.read_json('../input/jawiki-country.json', lines = True)
uk = df[df['title']=='イギリス'].text.values
ls = uk[0].split('\n')
d = {}
for line in ls:
    r = re.search(pattern, line)
    if r:
        d[r[1]]=r[2]
        
S = requests.Session()
URL = "https://commons.wikimedia.org/w/api.php"
PARAMS = {
    "action": "query",
    "format": "json",
    "titles": "File:" + d['国旗画像'],
    "prop": "imageinfo",
    "iiprop":"url"
}
R = S.get(url=URL, params=PARAMS)
DATA = R.json()
PAGES = DATA['query']['pages']
for k, v in PAGES.items():
    print (v['imageinfo'][0]['url'])

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
